In [4]:
from libMLP import ModMLP
from libMLP import fitMLP
from libMLP import SMAPE
from libMLP import MAPE
from libMLP import graphPrediction
from libMLP import plotHistory
import numpy as np
import pandas as pd

In [5]:
TS = pd.read_csv('TimesSeries\Electric_Production.csv')

En esta parte para las pruebas con ruido se pueden correr las funciones especiales del ruido.

In [ ]:
sigma = nSR(dB);
trainNoise(X_train, y_train, sigma, 2)

In [2]:
neuronas = [128,256,256,128]
activations = ['relu', 'relu', 'relu', 'sigmoid']

model = ModMLP(neuronas, activations, 7)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "forecasting"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 132,865
Trainable params: 

In [ ]:
history = fitMLP(model, X_train, y_train, X_val, y_val, X_test, y_test, 300)

In [ ]:
plotHistory(history)

In [ ]:
y_predic= model.predict(X_test)
Regresar a su dominio a y_test y y_predict

SMAPE(y_test, y_predict)
MAPE(y_test, y_predict)

In [ ]:
graphPrediction(y_test, y_predict)